In [4]:
import numpy as np
import pandas as pd
import qlib
from pathlib import Path
from qlib import data, model
from qlib.constant import REG_CN  # 'cn'
from qlib.data import D
from qlib.data.filter import ExpressionDFilter,NameDFilter
from qlib.utils import init_instance_by_config
from qlib.config import C

qlib.init(provider_uri={'day': Path("F:\qlib_bin")}, region=REG_CN)

[17320:MainThread](2023-07-15 19:28:50,488) INFO - qlib.Initialization - [config.py:415] - default_conf: client.
[17320:MainThread](2023-07-15 19:28:50,495) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[17320:MainThread](2023-07-15 19:28:50,495) INFO - qlib.Initialization - [__init__.py:76] - data_path={'day': WindowsPath('F:/qlib_bin')}


In [9]:
# calendar max and min
D.calendar(freq='day')[[-1,0]]

array([Timestamp('2023-07-14 00:00:00'), Timestamp('2005-01-04 00:00:00')],
      dtype=object)

In [27]:
# 只看沪深两市
sh = NameDFilter(name_rule_re = 'SH',fstart_time='2020-01-01', fend_time='2023-07-14')
sz = NameDFilter(name_rule_re = 'SZ',fstart_time='2020-01-01', fend_time='2023-07-14')

# 设置股票池
all_shsz = D.instruments(market='all', filter_pipe=[sh,sz])
shsz_dict = D.list_instruments(all_shsz).keys()
shsz_dict


dict_keys(['SZ000001', 'SZ000002', 'SZ000009', 'SZ000012', 'SZ000016', 'SZ000021', 'SZ000022', 'SZ000024', 'SZ000027', 'SZ000029', 'SZ000031', 'SZ000036', 'SZ000039', 'SZ000060', 'SZ000061', 'SZ000063', 'SZ000066', 'SZ000068', 'SZ000069', 'SZ000088', 'SZ000089', 'SZ000096', 'SZ000100', 'SZ000157', 'SZ000400', 'SZ000401', 'SZ000402', 'SZ000406', 'SZ000410', 'SZ000420', 'SZ000423', 'SZ000425', 'SZ000429', 'SZ000488', 'SZ000498', 'SZ000503', 'SZ000507', 'SZ000511', 'SZ000518', 'SZ000520', 'SZ000527', 'SZ000528', 'SZ000532', 'SZ000533', 'SZ000539', 'SZ000541', 'SZ000543', 'SZ000550', 'SZ000559', 'SZ000562', 'SZ000568', 'SZ000571', 'SZ000573', 'SZ000581', 'SZ000601', 'SZ000612', 'SZ000623', 'SZ000625', 'SZ000629', 'SZ000630', 'SZ000636', 'SZ000651', 'SZ000659', 'SZ000666', 'SZ000680', 'SZ000682', 'SZ000698', 'SZ000708', 'SZ000709', 'SZ000717', 'SZ000725', 'SZ000726', 'SZ000727', 'SZ000729', 'SZ000733', 'SZ000735', 'SZ000737', 'SZ000758', 'SZ000763', 'SZ000767', 'SZ000778', 'SZ000780', 'SZ00

In [29]:
# dataLoader to get data
"""
example_loader_config = {
    'feature': (['expr_1', 'expr_2', ... , 'expr_n'],
               ['name_1', 'name_2', ... , 'name_n']), # name list是经过_parse_fields_info()函数处理过才有的
    ...,
    'label':  (['expr_1', 'expr_2', ... , 'expr_n'],
               ['name_1', 'name_2', ... , 'name_n']),    
}
"""
from qlib.data.ops import Ts_Argmax


factor = (["Corr(Rank($high),Rank($volumn),3)"],[])
"($close-$open)/($high-$low+1e-12)"
label = ([],[])

data_loader_conf = {
    "class": "QlibDataLoader",
    "kwargs": {
        "config": {
            "feature": factor,
            "label": label,
        },
        "filter_pipe": [sh,sz],
        "freq": 'day',
        "inst_processor": inst_processor,
    },
}
data_loader = init_instance_by_config(data_loader_conf)

# # features, example from SH0000001
# D.features(instruments=all_shsz, start_time='2020-01-01', end_time='2023-07-14', fields=['$close', '$factor', '$high']).head(50)


# mkt_csi300 = D.instruments(market='all')
# D.features(mkt_csi300, fields, start_time=start, end_time=end, freq='day')

In [2]:
# 从calendar中读取 指定的频率和时间范围，查看数据的时间范围
start,end = '2018-01-01','2023-02-28'
cal_d = D.calendar(start_time=start, end_time=end, freq='day')
print(cal_d[:2])

# 初始化一个stook pool 从instruments中，指定 market 为文件名，读取个股code
mkt_csi300 = D.instruments(market='csi300')   # {'market': 'all', 'filter_pipe': []}
mkt_csi300_list = D.list_instruments(instruments=mkt_csi300, start_time=start, end_time=end, as_list=True)

# 有filter api 用来过滤数据
# expressionDFilter = ExpressionDFilter(rule_expression='$close>2000')
# instruments = D.instruments(market='csi300', filter_pipe=[expressionDFilter])
# D.list_instruments(instruments=instruments, start_time='2020-01-01', end_time='2020-09-25', as_list=True)


[Timestamp('2018-01-02 00:00:00') Timestamp('2018-01-03 00:00:00')]
